<a href="https://colab.research.google.com/github/jumbokh/Computer-Vision/blob/main/notebooks/%E5%86%92%E9%9A%AA08b_%E6%89%93%E9%80%A0_Stable_Diffusion_Chilloutmix_Web_APP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1. 安裝必要套件

In [1]:
!pip install transformers
!pip install diffusers["torch"]
!pip install accelerate
!pip install sentencepiece
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 96.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 15.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 38.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 17.6 

### 2. `pipeline` 一條龍服務

In [2]:
import torch

from diffusers import StableDiffusionPipeline

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [3]:
pipe = StableDiffusionPipeline.from_pretrained("stablediffusionapi/chilloutmix",
                                               torch_dtype=torch.float16)
pipe = pipe.to("cuda")

model_index.json:   0%|          | 0.00/584 [00:00<?, ?B/s]

text_encoder/model.safetensors not found


Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

safety_checker/config.json:   0%|          | 0.00/4.58k [00:00<?, ?B/s]

tokenizer/special_tokens_map.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

...

diffusion_pytorch_model.bin:   0%|          | 0.00/335M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

diffusion_pytorch_model.bin:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(


### 3. 關掉 NSFW 安全檢查

在公眾場合還是請大家不要執行。

In [4]:
pipe.safety_checker = lambda images, clip_input: (images, False)

### 4. 用 `gradio` 打造你的 Web App!

In [5]:
import gradio as gr

In [6]:
def text2image(prompt, negative_prompt):
    image = pipe(prompt, negative_prompt=negative_prompt).images[0]
    return image

In [7]:
inp1 = gr.Textbox(label="輸入你的咒語 (prompt)")
inp2 = gr.Textbox(label="負向的咒語 (negative prompt)")
outputs = gr.Image(label="生成圖")

可以自己設範例!

In [8]:
prompt = "masterpiece, best quality, 4k, photo, realistic, 1girl, kpop idol, smiling, (simple dress), sitting in a coffee shop, soft lips, looking at camera, (brown hair), beautiful detailed eyes, shallow depth of field, bokeh"
negative_prompt = "paintings, sketches, worst quality, low quality, normal quality, monochrome,  skin blemishes, age spots, ugly, duplicate, morbid, mutilated, mutated hands, poorly drawn hands, blurry, bad anatomy, bad proportions, extra limbs, disfigured, missing arms, extra legs, fused fingers, too many fingers, unclear eyes, bad hands, missing fingers, bad body, glans"

In [9]:
examples = [[prompt,
             negative_prompt]]

In [10]:
iface = gr.Interface(text2image,
                     inputs=[inp1, inp2],
                     outputs=outputs,
                     title="Stable Diffusion Chilloutmix 文字生圖 AI",
                     description="請輸入一段英文咒語 (prompt), 我會根據你的咒語畫出一張圖!",
                     examples=examples)

In [12]:
iface.launch(share=True)

Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://e989a14f62dc23339f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
